# Data preprocessing

In [7]:
# import area
import pandas as pd
import os


In [6]:
import os
import pandas as pd
# create dataframes using Path.cwd() to get the current working directory
# and os.path.join to join the current working directory with the file name to create the path
# and pd.read_csv to read the csv file and creat
# Get the current working directory
cwd = os.getcwd()

# Define the file path
file_path = os.path.join(cwd, 'gas.csv')

# Read the CSV file into a dataframe
gas = pd.read_csv(file_path)
power = pd.read_csv(os.path.join(cwd, 'power.csv'))
water = pd.read_csv(os.path.join(cwd, 'water.csv'))


# Exploratory for water

In [12]:
# function to describe and info for the dataframes

def describe_info(df):
    print(df.describe())
    print(df.info())
    print(df.head())

# call the function to describe and info for the dataframes
describe_info(water)


             days        cons      av_day  days_month   days_cons  \
count  123.000000  123.000000  182.000000  192.000000  192.000000   
mean    30.853659   10.192780    0.270124   30.453125   24.192708   
std     29.025055   15.932932    0.106032    0.771135   10.301189   
min      0.000000    0.000000    0.000000   28.000000    2.000000   
25%     16.000000    4.661500    0.246865   30.000000   18.750000   
50%     29.000000    8.002000    0.276644   31.000000   30.000000   
75%     31.000000    9.528000    0.312021   31.000000   31.000000   
max    200.000000  108.000000    0.821467   31.000000   31.000000   

       missing_days   calc_cons   diff_cons  cons_month  
count    192.000000  182.000000  123.000000  182.000000  
mean       6.260417    8.175928    1.670592    8.218311  
std       10.216243    3.158706   15.879453    3.232844  
min        0.000000    0.000000  -13.143467    0.000000  
25%        0.000000    7.378803   -3.200462    7.349209  
50%        0.000000    8.33717

In [11]:
# create new column in df water and power with the name 'cons_month' with the value of av_day * days_cons + that next month av_day * missing_days

water['cons_month'] = water['av_day'] * water['days_cons'] + water['av_day'] * water['missing_days'] 

# SQL DB